## property: uniform interface attribute with additional processing

In [7]:
import math

class Circle:
    def __init__(self, rad):
        self.rad = rad
    
    @property
    def rad(self):
        return self._rad
    
    @rad.setter
    def rad(self, val):
        if type(val) not in [float, int]:
            raise TypeError
        self._rad = val
    
    
    
    @property
    def perimeter(self):
        return math.pi * self.rad * 2
    
    @property
    def area(self):
        import math
        return math.pi * self.rad ** 2

c = Circle(2)
print(c.rad)
print(c.perimeter)
print(c.area)
c.rad = '123'

2
12.566370614359172
12.566370614359172


TypeError: 

## super() follows \_\_mro__

In [20]:
# mro uses DFS

from pprint import pprint


class BaseA:
    pass

class BaseB:
    pass

class A(BaseA):
    pass

class B(BaseB):
    pass

class C(A,B):
    pass

pprint(C.__mro__)



class Base:
    pass

class A(Base):
    pass

class B(Base):
    pass

class C(A,B):
    pass

pprint(C.__mro__)
pprint(A.__mro__)

(<class '__main__.C'>,
 <class '__main__.A'>,
 <class '__main__.BaseA'>,
 <class '__main__.B'>,
 <class '__main__.BaseB'>,
 <class 'object'>)
(<class '__main__.C'>,
 <class '__main__.A'>,
 <class '__main__.B'>,
 <class '__main__.Base'>,
 <class 'object'>)
(<class '__main__.A'>, <class '__main__.Base'>, <class 'object'>)


In [34]:
# super() lets inheritance hierarchy to follow on initial mro sequence

class A:
    def foo(self):
        print('A')
        super().foo()
        
class B:
    def foo(self):
        print('B')
        
        
class C(A,B):
    pass

c = C()
print(C.__mro__)
c.foo()
# A
# B (this is not shown when super() is not implemented in class 'A')

# since inheritance cursor will follow mro sequence, super() in class 'B' will not work
# this is because class 'object' does not have method 'foo'

(<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class 'object'>)
A
B


In [45]:
# (probably)
# inheritance means using parents' namespace
# within logic above, B does not have bar(), but can assess bar() in A
# to manually enable this, you use super().bar()

class A:
    def foo(self):
        print('foo', self.__class__)
        
    def bar(self):
        print('bar', self.__class__)
        
class B(A):
    
    def foo(self):
        super().foo()

        
    def hoi(self):
        super().bar()
        
b = B()
b.foo()
b.bar()
b.hoi()

foo <class '__main__.B'>
bar <class '__main__.B'>
bar <class '__main__.B'>


## descriptor class

In [50]:
# descriptor class requires __get__(self, instance) and __set__(self, instance, value)
# above gets and sets attribute to __dict__ of each second parameter, namely instance
# the key of the attribute should be initialized beforehand

class Descriptor:
    def __init__(self, key):
        self.key = key
        
    def __get__(self, inst):
        return inst.__dict__[self.key]
    
    def __set__(self, inst, val):
        print('warning, new attribute added')   # doing sth meaningful
        inst.__dict__[self.key] = val
    
    def __delete__(self, inst):
        print('warning, attribute deleted')   # doing sth meaningful
        del inst.__dict__[self.key]
        
class Class:
    a = Descriptor('a')
    b = Descriptor('b')
    
    def __init__(self, a, b):
        self.a = a
        self.b = b

c = Class(1,2)
print(c.__dict__)
del c.a
del c.b

warning, new attribute added
warning, new attribute added
{'a': 1, 'b': 2}
warning, attribute deleted
warning, attribute deleted
